In [27]:
from google.colab import files
import pandas as pd
uploads = files.upload()

Saving Budget_Analysis.csv to Budget_Analysis (1).csv


In [28]:
df = pd.read_csv('Budget_Analysis.csv')
df.head()

,date,category,amount
0,2022-07-06 05:57:10 +0000,Restuarant,5.50
1,2022-07-06 05:57:27 +0000,Market,2.00
2,2022-07-06 05:58:12 +0000,Coffe,30.10
3,2022-07-06 05:58:25 +0000,Market,17.33
4,2022-07-06 05:59:00 +0000,Restuarant,5.50


In [29]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['category'] = le.fit_transform(df['category'])
df.head()

,date,category,amount
0,2022-07-06 05:57:10 +0000,14,5.50
1,2022-07-06 05:57:27 +0000,9,2.00
2,2022-07-06 05:58:12 +0000,2,30.10
3,2022-07-06 05:58:25 +0000,9,17.33
4,2022-07-06 05:59:00 +0000,14,5.50


In [30]:
Q1 = df['amount'].quantile(0.25)
Q3 = df['amount'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df[(df['amount'] >= lower_bound) & (df['amount'] <= upper_bound)]

In [31]:
df['date'] = pd.to_datetime(df['date'])
df['Year'] = df['date'].dt.year
df['Month'] = df['date'].dt.month
df['Month_Name'] = df['date'].dt.month_name()
df['Day'] = df['date'].dt.day
df['Day_Name'] = df['date'].dt.day_name()
df['Weekday'] = df['date'].dt.weekday

In [32]:
import numpy as np
df['amount'] = df['amount'].abs()   # Ensure no negative values
df['Log_Amount'] = np.log1p(df['amount'])  # Helps ML models
df['High_Expense'] = df['amount'].apply(lambda x: 1 if x > df['amount'].mean() else 0)

In [33]:
df.head()

,date,category,amount,Year,Month,Month_Name,Day,Day_Name,Weekday,Log_Amount,High_Expense
0,2022-07-06 05:57:10+00:00,14,5.50,2022,7,July,6,Wednesday,2,1.871802,0
1,2022-07-06 05:57:27+00:00,9,2.00,2022,7,July,6,Wednesday,2,1.098612,0
3,2022-07-06 05:58:25+00:00,9,17.33,2022,7,July,6,Wednesday,2,2.908539,1
4,2022-07-06 05:59:00+00:00,14,5.50,2022,7,July,6,Wednesday,2,1.871802,0
5,2022-07-06 05:59:15+00:00,9,11.78,2022,7,July,6,Wednesday,2,2.547881,1


In [34]:
df['Year_Month'] = df['date'].dt.to_period('M')
monthly_stats = df.groupby('Year_Month')['amount'].agg(
    Monthly_Total='sum',
    Monthly_Avg='mean',
    Monthly_Max='max'
)

/tmp/ipython-input-3457518955.py:1: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['Year_Month'] = df['date'].dt.to_period('M')


In [35]:
df = df.merge(monthly_stats, on='Year_Month', how='left')

In [36]:
print("Feature Engineered Dataset Shape:", df.shape)
df.head()

Feature Engineered Dataset Shape: (4155, 15)


,date,category,amount,Year,Month,Month_Name,Day,Day_Name,Weekday,Log_Amount,High_Expense,Year_Month,Monthly_Total,Monthly_Avg,Monthly_Max
0,2022-07-06 05:57:10+00:00,14,5.50,2022,7,July,6,Wednesday,2,1.871802,0,2022-07,544.13,7.663803,21.0
1,2022-07-06 05:57:27+00:00,9,2.00,2022,7,July,6,Wednesday,2,1.098612,0,2022-07,544.13,7.663803,21.0
2,2022-07-06 05:58:25+00:00,9,17.33,2022,7,July,6,Wednesday,2,2.908539,1,2022-07,544.13,7.663803,21.0
3,2022-07-06 05:59:00+00:00,14,5.50,2022,7,July,6,Wednesday,2,1.871802,0,2022-07,544.13,7.663803,21.0
4,2022-07-06 05:59:15+00:00,9,11.78,2022,7,July,6,Wednesday,2,2.547881,1,2022-07,544.13,7.663803,21.0
